In [ ]:
# 读取fileList
import numpy as np

fileList = np.load("/homegys/lab/DeepFold/protein_infor/entPathArray.npy", allow_pickle=True).tolist()

In [ ]:
# # GitHub 原代码 生成距离矩阵
# import sys
# import os
# import numpy
# import scipy
# import scipy.spatial

# from Bio.PDB import PDBParser


# def get_distance_matrix(pdb_path):
#     parser = PDBParser()
#     structure = parser.get_structure('structure', pdb_path).get_list()[0]
#     residue_positions = get_residue_positions(structure)
#     pdb_dist_mat = scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(residue_positions, 'euclidean'))
#     pdb_dist_mat[numpy.isnan(pdb_dist_mat)] = float('inf')
#     return pdb_dist_mat


# def get_residue_ids(structure):
#     ids = [r.get_id()[1] for r in structure.get_residues()]
#     return ids


# def get_residue_positions(structure):
#     residue_ids = get_residue_ids(structure)
#     positions = numpy.ones((residue_ids[-1] - residue_ids[0] + 1, 3)) * float('inf')
#     for residue in structure.get_residues():
#         atoms = residue.get_atoms()
#         for a in atoms:
#             if a.get_name() == 'CA':
#                 positions[residue.get_id()[1] - residue_ids[0]] = a.get_coord()

#     return positions

In [ ]:
# 修改的 GitHub 原代码 生成距离矩阵
import sys
import os
import numpy
import scipy
import scipy.spatial

from Bio.PDB import PDBParser


def get_distance_matrix_modified(pdb_path):
    parser = PDBParser()
    structure = parser.get_structure('structure', pdb_path).get_list()[0]
    residue_positions = get_residue_positions(structure)
    pdb_dist_mat = scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(residue_positions, 'euclidean'))
    pdb_dist_mat[numpy.isnan(pdb_dist_mat)] = float('inf')
    return pdb_dist_mat


def get_residue_ids(structure):
    ids = [r.get_id()[1] for r in structure.get_residues()]
    return ids


def get_residue_positions(structure):
    residue_ids = get_residue_ids(structure)
    positions = numpy.ones((len(residue_ids), 3)) * float('inf')
    i = 0
    for residue in structure.get_residues():
        atoms = residue.get_atoms()
        for a in atoms:
            if a.get_name() == 'CA':
                positions[i] = a.get_coord()
                i = i + 1
    return positions

In [13]:
# 自己解析 ent文件 生成距离矩阵
import sys
import os
import numpy
import scipy
import scipy.spatial

def get_distance_matrix_mine(pdb_path):
    CA_positions = []
    with open(pdb_path, 'r') as f:
        while True:
            line = f.readline()
            if not line:
                break
            if line[:4] != "ATOM":
                if line[:6] != "HETATM":
                    continue
            if line[13:15] == "CA":
                position = [float(line[30:38].strip()), float(line[38:46].strip()), float(line[46:54].strip())]
                CA_positions.append(position)
    pdb_dist_mat = scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(CA_positions, 'euclidean'))
    return pdb_dist_mat

In [ ]:
# 生成14000多个距离矩阵并保存
import warnings
warnings.filterwarnings('ignore')
import numpy as np

errorList = []
errorFile = []

# save_dir = "/homegys/lab/DeepFold/distance_matrix/distance_matrix_inf/"
save_dir = "/homegys/lab/DeepFold/distance_matrix/distance_matrix_mine/"
# save_dir = "/homegys/lab/DeepFold/distance_matrix/distance_matrix_modified/"

for file_n in fileList:
    ID = file_n.split("/")[-1].split(".")[0]
    try:
        # M = get_distance_matrix(file_n)
        M = get_distance_matrix_mine(file_n)
        # M = get_distance_matrix_modified(file_n)
    except:
        errorList.append(ID)
        errorFile.append(file_n)
        print("error: " + ID)
    else:
        np.save(save_dir+ID+".npy", M)
        # print("done: " + ID)

In [ ]:
print(len(errorList))
print(len(errorFile))

In [ ]:
np.save("/homegys/lab/DeepFold/protein_infor/errorIDArrayMine.npy", errorList)
np.save("/homegys/lab/DeepFold/protein_infor/errorEntPathArrayMine.npy", errorFile)

In [ ]:
errorList = ['d1nria_', 'd3msza1', 'd1ngvb_', 'd1bd7a_', 'd4ax1b_', 'd1nova_']
errorFile = ['/home/wngys/scop/scope-2.07-40/nr/d1nria_.ent', 
'/home/wngys/scop/scope-2.07-40/ms/d3msza1.ent', 
'/home/wngys/scop/scope-2.07-40/ng/d1ngvb_.ent', 
'/home/wngys/scop/scope-2.07-40/bd/d1bd7a_.ent', 
'/home/wngys/scop/scope-2.07-40/ax/d4ax1b_.ent', 
'/home/wngys/scop/scope-2.07-40/no/d1nova_.ent']

np.save("/homegys/lab/DeepFold/protein_infor/errorIDArrayInf.npy", errorList)
np.save("/homegys/lab/DeepFold/protein_infor/errorEntPathArrayInf.npy", errorFile)

In [ ]:
# 测试：加载举例矩阵
M_2 = np.load("/homegys/lab/DeepFold/distance_matrix/distance_matrix_mine/d1e2aa_.npy", allow_pickle=True)
print(M_2.shape)
print(M_2)

In [ ]:
%pip install biopython